In [ ]:
from mxnet import ndarray as nd
from mxnet import gluon
from mxnet import autograd

num_train = 100
num_test = 100
true_W = [1.2,-3.4,5.6]
true_b = 5.0

In [ ]:
x = nd.random_normal(shape=(num_train+num_test,1))
X = nd.concat(x,nd.power(x,2),nd.power(x,3))
y = true_W[0]*X[:,0] + true_W[1]*X[:,1] + true_W[2]*X[:,2] + true_b
y += .1 * nd.random_normal(shape=y.shape)
y_train,y_test = y[:num_train],y[num_train:]

('x:',x[:5],'X:',X[:5],'y:',y[:5])

In [ ]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt
from utils import square_loss

def test(net,X,y):
    return square_loss(net(X),y).mean().asscalar()

def train(X_train,X_test,y_train,y_test):
    #线性回归模型
    net = gluon.nn.Sequential()
    with net.name_scope():
        net.add(gluon.nn.Dense(1))
    net.initialize()
    learning_rate = 0.01
    epochs = 100
    #batch_size = min(1,y_train.shape[0])
    batch_size = 2
    dataset_train = gluon.data.ArrayDataset(X_train,y_train)
    data_iter_train = gluon.data.DataLoader(dataset_train,batch_size,shuffle=True)
    #默认SGD和均方误差
    trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':learning_rate})
    square_loss = gluon.loss.L2Loss()
    #保存训练和测试损失
    train_loss = []
    test_loss = []
    for e in range(epochs):
        for data,label in data_iter_train:
            with autograd.record():
                output = net(data)
                loss = square_loss(output,label)
            loss.backward()
            trainer.step(batch_size)
        train_loss.append(square_loss(net(X_train),y_train).mean().asscalar())
        test_loss.append(square_loss(net(X_test),y_test).mean().asscalar())
    #打印结果
    plt.plot(train_loss)
    plt.plot(test_loss)
    plt.legend(['train','test'])
    plt.show()
    return ('learned weight',net[0].weight.data(),'learned bias',net[0].bias.data())

In [ ]:
train(X[:num_train,:],X[num_train:,:],y[:num_train],y[num_train:])

In [ ]:
train(x[:num_train,:],x[num_train:,:],y[:num_train],y[num_train:])

In [ ]:
train(X[0:2,:],X[num_train:,:],y[0:2],y[num_train:])